In [61]:
import yfinance as yf
import datetime as dt
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
def black_scholes_with_greeks(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                 - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                 + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100  # per 1% change in vol

    return {
        'Price': price,
        'Delta': delta,
        'Gamma': gamma,
        'Theta (per day)': theta,
        'Vega (per 1%)': vega,
        'Rho (per 1%)': rho
    }

# Parameters
S = 100        # Current stock price
K = 150        # Strike price
T = 365 / 365   # Time to maturity (in years)
r = 0.05       # Annual risk-free rate
sigma = 0.3    # Volatility
option_type = 'put'

# Compute and display
results = black_scholes_with_greeks(S, K, T, r, sigma, option_type)

for greek, value in results.items():
    print(f"{greek}: {value:.2f}")

Price: 44.74
Delta: -0.85
Gamma: 0.01
Theta (per day): 0.01
Vega (per 1%): 0.23
Rho (per 1%): -1.30
